In [1]:
#importing the necessary packages
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from missingpy import MissForest
from fancyimpute import IterativeImputer
from mlens.ensemble import SuperLearner
from imblearn.over_sampling import SMOTE

Using TensorFlow backend.
[MLENS] backend: threading


In [2]:
data=pd.read_csv("TrainingData.csv") #reading the test and train data
data2=pd.read_csv("testX.csv")

C:\ProgramData\Anaconda3\lib\site-packages\IPython\core\interactiveshell.py:3049: DtypeWarning: Columns (19) have mixed types. Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


In [3]:
Y = data.loc[:, data.columns == 'default_ind']  #separating out the Y values
app_key_train = data.loc[:, data.columns == 'application_key'] #separating out the application keys
data = data.loc[:, data.columns != 'default_ind']
data= data.loc[:, data.columns != 'application_key'] #separating out the application keys
app_key_test = data2.loc[:, data2.columns == 'application_key']
data2= data2.loc[:, data2.columns != 'application_key']
df = pd.concat([data, data2], ignore_index=True)

In [4]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 130000 entries, 0 to 129999
Data columns (total 48 columns):
mvar1     130000 non-null object
mvar2     120787 non-null float64
mvar3     129171 non-null float64
mvar4     129171 non-null float64
mvar5     129171 non-null float64
mvar6     130000 non-null object
mvar7     130000 non-null object
mvar8     130000 non-null object
mvar9     130000 non-null object
mvar10    130000 non-null object
mvar11    130000 non-null object
mvar12    130000 non-null object
mvar13    130000 non-null object
mvar14    130000 non-null int64
mvar15    130000 non-null object
mvar16    130000 non-null object
mvar17    130000 non-null object
mvar18    130000 non-null object
mvar19    130000 non-null object
mvar20    130000 non-null object
mvar21    93158 non-null float64
mvar22    81964 non-null float64
mvar23    63712 non-null float64
mvar24    99204 non-null float64
mvar25    130000 non-null object
mvar26    130000 non-null object
mvar27    130000 non-null ob

In [5]:
print(df)

       mvar1   mvar2   mvar3  mvar4  mvar5    mvar6    mvar7    mvar8  \
0       1696  1.6541   0.000  0.000  0.000        0     6015      322   
1       1846  0.8095   0.000  0.000  0.000      102     7532     3171   
2       1745  0.4001   0.000  0.000  0.000  missing     2536  missing   
3       1739  0.2193   0.000  0.000  0.000     1982    26440     4955   
4       1787  0.0118   0.225  0.000  0.000     5451     5494     5494   
5       1579     NaN   3.502  0.000  0.000  missing  missing  missing   
6       1818  0.4001   0.000  0.000  0.000  missing     1088  missing   
7         na     NaN     NaN    NaN    NaN  missing  missing  missing   
8       1836  0.1358   0.000  0.000  0.000      347    38964    17828   
9       1839  0.1981   0.000  0.000  0.000      793     6131     6045   
10      1903  0.0000   0.000  0.000  0.000       22    19518     9910   
11      1681  1.5888   5.685  3.566  0.000       93     3554      439   
12      1841  0.5388   0.000  0.000  0.000     1487

In [6]:
df = df.replace({'missing': np.NaN}, regex=True)
df = df.replace({'na': np.NaN}, regex=True)
df = df.replace({'N/A': np.NaN}, regex=True)
df = df.replace({'L': 0}, regex=True) #encoding the categorical variable
df = df.replace({'C': 1}, regex=True)

In [7]:
print(df)

       mvar1   mvar2   mvar3  mvar4  mvar5  mvar6   mvar7   mvar8   mvar9  \
0       1696  1.6541   0.000  0.000  0.000      0    6015     322   40369   
1       1846  0.8095   0.000  0.000  0.000    102    7532    3171   18234   
2       1745  0.4001   0.000  0.000  0.000    NaN    2536     NaN     NaN   
3       1739  0.2193   0.000  0.000  0.000   1982   26440    4955   20316   
4       1787  0.0118   0.225  0.000  0.000   5451    5494    5494    7987   
5       1579     NaN   3.502  0.000  0.000    NaN     NaN     NaN     NaN   
6       1818  0.4001   0.000  0.000  0.000    NaN    1088     NaN    1536   
7        NaN     NaN     NaN    NaN    NaN    NaN     NaN     NaN     NaN   
8       1836  0.1358   0.000  0.000  0.000    347   38964   17828   70729   
9       1839  0.1981   0.000  0.000  0.000    793    6131    6045   48959   
10      1903  0.0000   0.000  0.000  0.000     22   19518    9910   90618   
11      1681  1.5888   5.685  3.566  0.000     93    3554     439    2527   

In [8]:
df.drop(['mvar23','mvar30','mvar31','mvar35','mvar40','mvar41','mvar45'], axis=1) # droppping the missing columns

,mvar1,mvar2,mvar3,mvar4,mvar5,mvar6,mvar7,mvar8,mvar9,mvar10,...,mvar36,mvar37,mvar38,mvar39,mvar42,mvar43,mvar44,mvar46,mvar47,mvar48
0,1696,1.6541,0.000,0.000,0.000,0,6015,322,40369,18414,...,8,10,4,1,0.08696,10,0.63899,0,1,10
1,1846,0.8095,0.000,0.000,0.000,102,7532,3171,18234,13664,...,8,0,2,0,0,13,0.63836,NaN,0,732
2,1745,0.4001,0.000,0.000,0.000,NaN,2536,NaN,NaN,2536,...,1,NaN,1,0,0,1,1.00000,0,1,89
3,1739,0.2193,0.000,0.000,0.000,1982,26440,4955,20316,37013,...,3,3,2,0,0.15385,3,0.53241,0,0,3
4,1787,0.0118,0.225,0.000,0.000,5451,5494,5494,7987,4696,...,2,3,2,0,0,1,0.92665,NaN,0,5
5,1579,NaN,3.502,0.000,0.000,NaN,NaN,NaN,NaN,0,...,2,NaN,2,NaN,1.5,0,NaN,NaN,1,35
6,1818,0.4001,0.000,0.000,0.000,NaN,1088,NaN,1536,1498,...,2,0,0,0,0,2,0.87224,0,1,2
7,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1,2
8,1836,0.1358,0.000,0.000,0.000,347,38964,17828,70729,65843,...,6,12,2,0,0,10,0.89868,0,0,5
9,1839,0.1981,0.000,0.000,0.000,793,6131,6045,48959,31640,...,10,7,0,0,0.08824,14,0.33834,0,0,3247


In [9]:
imputer=MissForest()  #imputing using miss forest and separating the test and train data set
df1=imputer.fit_transform(df)
df=pd.DataFrame(df1)
data, data2 = df[:83000, :], df[83000:, :]

In [11]:
data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 83000 entries, 0 to 82999
Data columns (total 41 columns):
mvar1     83000 non-null float64
mvar2     83000 non-null float64
mvar3     83000 non-null float64
mvar4     83000 non-null float64
mvar5     83000 non-null float64
mvar6     83000 non-null float64
mvar7     83000 non-null float64
mvar8     83000 non-null float64
mvar9     83000 non-null float64
mvar10    83000 non-null float64
mvar11    83000 non-null float64
mvar12    83000 non-null float64
mvar13    83000 non-null float64
mvar14    83000 non-null int64
mvar15    83000 non-null float64
mvar16    83000 non-null float64
mvar17    83000 non-null float64
mvar18    83000 non-null float64
mvar19    83000 non-null int64
mvar20    83000 non-null float64
mvar21    83000 non-null float64
mvar22    83000 non-null float64
mvar24    83000 non-null float64
mvar25    83000 non-null float64
mvar26    83000 non-null float64
mvar27    83000 non-null float64
mvar28    83000 non-null float64
mvar

In [12]:
data2.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 47000 entries, 0 to 46999
Data columns (total 41 columns):
mvar1     47000 non-null float64
mvar2     47000 non-null float64
mvar3     47000 non-null float64
mvar4     47000 non-null float64
mvar5     47000 non-null float64
mvar6     47000 non-null float64
mvar7     47000 non-null float64
mvar8     47000 non-null float64
mvar9     47000 non-null float64
mvar10    47000 non-null float64
mvar11    47000 non-null float64
mvar12    47000 non-null float64
mvar13    47000 non-null float64
mvar14    47000 non-null int64
mvar15    47000 non-null float64
mvar16    47000 non-null float64
mvar17    47000 non-null float64
mvar18    47000 non-null float64
mvar19    47000 non-null int64
mvar20    47000 non-null float64
mvar21    47000 non-null float64
mvar22    47000 non-null float64
mvar24    47000 non-null float64
mvar25    47000 non-null float64
mvar26    47000 non-null float64
mvar27    47000 non-null float64
mvar28    47000 non-null float64
mvar

In [13]:
# Splitting the data
from sklearn.model_selection import train_test_split
X_train,X_test,Y_train,Y_test=train_test_split(data,Y,test_size=0.2,random_state=0)


In [14]:
# feature Scaling
from sklearn.preprocessing import StandardScaler
sc_x=StandardScaler()
X_train=sc_x.fit_transform(X_train)
X_test=sc_x.transform(X_test)

In [15]:
#Over_sampling

smt=SMOTE()
X_train,Y_train=smt.fit_sample(X_train,Y_train)
# XGB classifier
from xgboost import XGBClassifier
classifier=XGBClassifier(n_estimators=200)


C:\ProgramData\Anaconda3\lib\site-packages\sklearn\utils\validation.py:724: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)


In [16]:
# Logistic Regression Classifier
from sklearn.linear_model import LogisticRegression
classifier1=LogisticRegression(random_state=0)

In [17]:
#random forest classifier
from sklearn.ensemble import RandomForestClassifier
classifier2=RandomForestClassifier()
#SVM
from sklearn.svm import SVC
#AdaBoost Classifier
from sklearn.ensemble import AdaBoostClassifier
adaboost=AdaBoostClassifier()
#NaiveBayes 
from sklearn.naive_bayes import GaussianNB
naivebayes=GaussianNB()
#ensemble

In [18]:
#Voting
from sklearn.ensemble import VotingClassifier
model=VotingClassifier(estimators=[('lr',classifier1),('xgb',classifier),('xgb1',classifier),('xgb2',classifier),('rf',classifier2),('nb1',naivebayes),('rf1',classifier2),('ad1',adaboost),
                                   ('xgb3',classifier),('xgb4',classifier)],voting='hard')
model.fit(X_train,Y_train)

C:\ProgramData\Anaconda3\lib\site-packages\sklearn\linear_model\logistic.py:432: FutureWarning: Default solver will be changed to 'lbfgs' in 0.22. Specify a solver to silence this warning.
  FutureWarning)
C:\ProgramData\Anaconda3\lib\site-packages\sklearn\ensemble\forest.py:245: FutureWarning: The default value of n_estimators will change from 10 in version 0.20 to 100 in 0.22.
  "10 in version 0.20 to 100 in 0.22.", FutureWarning)
C:\ProgramData\Anaconda3\lib\site-packages\sklearn\ensemble\forest.py:245: FutureWarning: The default value of n_estimators will change from 10 in version 0.20 to 100 in 0.22.
  "10 in version 0.20 to 100 in 0.22.", FutureWarning)


VotingClassifier(estimators=[('lr',
                              LogisticRegression(C=1.0, class_weight=None,
                                                 dual=False, fit_intercept=True,
                                                 intercept_scaling=1,
                                                 l1_ratio=None, max_iter=100,
                                                 multi_class='warn',
                                                 n_jobs=None, penalty='l2',
                                                 random_state=0, solver='warn',
                                                 tol=0.0001, verbose=0,
                                                 warm_start=False)),
                             ('xgb',
                              XGBClassifier(base_score=0.5, booster='gbtree',
                                            colsample_bylevel=1,
                                            colsample...
                                            colsample_byn

In [21]:
#prediction

X=data2.iloc[:].values
from sklearn.preprocessing import StandardScaler
sc_x=StandardScaler()
X=sc_x.fit_transform(X)
#prediction with voting
y_pred=model.predict(X)

In [26]:
#prediction


#y_pred = pd.DataFrame(y_pred, columns=['y_pred']).to_csv('y_pred.csv')

# confusion matrix

from sklearn.metrics import confusion_matrix
y_test_pred=model.predict(X_test)
cm=confusion_matrix(Y_test,y_test_pred)
print(cm)
a=cm[0][0]+cm[1][1]
b=cm[0][0]+cm[0][1]+cm[1][0]+cm[1][1]
print(a/b)

from sklearn.metrics import balanced_accuracy_score
from sklearn.metrics import f1_score


ans=balanced_accuracy_score(Y_test,y_test_pred)
print(ans)

f1=f1_score(Y_test,y_test_pred)
print(f1,'f1')

[[8676 3080]
 [1591 3253]]
0.7186144578313253
0.7047792802677284
0.5820882168739375 f1
